In [7]:
!unzip kfp_eval_samples.zip -d /kaggle/input/files-kfp/kfp_eval_samples

unzip:  cannot find or open kfp_eval_samples.zip, kfp_eval_samples.zip.zip or kfp_eval_samples.zip.ZIP.


In [19]:
!pip install -q kfp


In [20]:
from pathlib import Path

output_dir = Path("/kaggle/input/grok-kfp-data/kfp_eval_samples")
file_count = len(list(output_dir.glob("*")))  # Counts all files
print(f"📁 Total files in directory: {file_count}")


📁 Total files in directory: 9


In [21]:
!pip install pytest


In [25]:
import importlib.util
import traceback
import logging
import os
from pathlib import Path
from typing import Callable, Optional
import pytest
from kfp import dsl, compiler

# --- Logging setup ---
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# --- Directory with generated .py files ---
PIPELINE_DIR = Path("/kaggle/input/enhanced-data-cht/kfp_eval_samples_2")

# Validate directory existence
if not PIPELINE_DIR.exists():
    raise FileNotFoundError(f"Pipeline directory {PIPELINE_DIR} does not exist")

pipeline_files = list(PIPELINE_DIR.glob("*.py"))
if not pipeline_files:
    raise FileNotFoundError(f"No .py files found in {PIPELINE_DIR}")


# === Function to load @dsl.pipeline ===
def get_pipeline_func(py_file: Path) -> Optional[Callable]:
    """Load a pipeline function decorated with @dsl.pipeline."""
    try:
        spec = importlib.util.spec_from_file_location(py_file.stem, py_file)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)

        funcs = [
            obj for obj in vars(module).values()
            if callable(obj) and hasattr(obj, "_component_spec")  # <- this marks @dsl.pipeline
        ]
        return funcs[0] if funcs else None
    except Exception as e:
        logger.error(f"Error loading {py_file.name}: {e}")
        traceback.print_exc()
        return None


# === Function to validate presence of @component decorators ===
def validate_component_decorators(module) -> bool:
    """Check if module contains at least one valid @component-decorated function."""
    try:
        components = [
            obj for obj in vars(module).values()
            if callable(obj) and hasattr(obj, "_component_human_name")
        ]
        if not components:
            logger.warning(f"⚠️ No @component found in {getattr(module, '__file__', 'unknown module')}")
        return True
    except Exception as e:
        logger.error(f"Error validating components: {e}")
        return False


# === Optional check for external dependencies ===
def check_external_dependencies(py_file: Path) -> bool:
    """Detect hardcoded cloud dependencies in the code."""
    with open(py_file, 'r') as f:
        content = f.read()
    keywords = ['snowflake', 'google.cloud', 'bigquery', 'gs://']
    return any(kw in content.lower() for kw in keywords)


# === TEST 1: Pipeline Compilation ===
@pytest.mark.timeout(60)
@pytest.mark.parametrize("py_file", pipeline_files)
def test_pipeline_compilation(py_file: Path) -> None:
    """Test that the pipeline compiles successfully to YAML."""
    pipeline_func = get_pipeline_func(py_file)
    if pipeline_func is None:
        pytest.skip(f"⏭️ No @dsl.pipeline function found in {py_file.name}")

    # Load module and validate components
    spec = importlib.util.spec_from_file_location(py_file.stem, py_file)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)

    assert validate_component_decorators(module), f"Invalid @component decorators in {py_file.name}"

    if check_external_dependencies(py_file):
        logger.warning(f"⚠️ Skipping {py_file.name} due to cloud dependencies")
        pytest.skip(f"{py_file.name} has external services")

    yaml_path = str(py_file.with_suffix(".yaml"))
    try:
        compiler.Compiler().compile(pipeline_func, yaml_path)
        assert os.path.exists(yaml_path), f"YAML not created for {py_file.name}"
        logger.info(f"✅ Compiled {py_file.name} → {yaml_path}")
    except Exception as e:
        logger.error(f"❌ Compilation failed: {e}")
        traceback.print_exc()
        pytest.fail(f"Compilation failed for {py_file.name}: {e}")
    finally:
        if os.path.exists(yaml_path):
            os.remove(yaml_path)
            logger.info(f"🧹 Cleaned up {yaml_path}")


# === TEST 2: Parameter Type Validation ===
@pytest.mark.parametrize("py_file", pipeline_files)
def test_pipeline_parameters(py_file: Path) -> None:
    """Validate parameter types for @dsl.pipeline functions."""
    pipeline_func = get_pipeline_func(py_file)
    if pipeline_func is None:
        pytest.skip(f"⏭️ No @dsl.pipeline function found in {py_file.name}")

    import inspect
    sig = inspect.signature(pipeline_func)
    valid_types = (str, int, float, bool, list, dict)

    for param_name, param in sig.parameters.items():
        param_type = param.annotation
        if param_type is inspect.Parameter.empty:
            logger.warning(f"⚠️ Parameter '{param_name}' in {py_file.name} lacks type annotation")
        elif not (param_type in valid_types or isinstance(param_type, type)):
            pytest.fail(f"❌ Invalid type '{param_type}' for parameter '{param_name}' in {py_file.name}")
        logger.info(f"✅ {py_file.name}: Validated param '{param_name}' ({param_type})")


In [26]:
!pytest test_kfp_pipelines.py --maxfail=10 --disable-warnings -q


FFFFFFFFFF
=================================== FAILURES ===================================
_____________________ test_pipeline_compilation[py_file0] ______________________

py_file = PosixPath('/kaggle/input/files-kfp/kfp_eval_samples/muhammadyaseen__kubeflow-on-linode__minio_census_pipeline.py')

    @pytest.mark.parametrize("py_file", pipeline_files)
    def test_pipeline_compilation(py_file):
        pipeline_func = get_pipeline_func(py_file)
>       assert pipeline_func is not None, f"No @dsl.pipeline function found in {py_file.name}"
E       AssertionError: No @dsl.pipeline function found in muhammadyaseen__kubeflow-on-linode__minio_census_pipeline.py
E       assert None is not None

test_kfp_pipelines.py:30: AssertionError
_____________________ test_pipeline_compilation[py_file1] ______________________

py_file = PosixPath('/kaggle/input/files-kfp/kfp_eval_samples/s102401002__kubeflowPipeline0722__kubeflowPipeline0722.py')

    @pytest.mark.parametrize("py_file", pipeline_files)